<a href="https://colab.research.google.com/github/alexf05/pclp3/blob/main/grile_UMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#FECHET Alex-Ciprian
# alex_ciprian.fechet@stud.acs.upb.ro

In [47]:
#@title get data from pdf
# Instalează PyMuPDF dacă nu e deja instalat
!pip install --upgrade pymupdf

import re
import fitz  # PyMuPDF
import pandas as pd
from google.colab import files

# Funcție de upload a fișierului PDF
print("Încarcă fișierul PDF cu grilele pentru examen:")
uploaded = files.upload()

# Preluăm primul fișier uploadat (presupunem că ai încărcat un singur PDF)
file_name = list(uploaded.keys())[0]

# Deschidem documentul PDF
doc = fitz.open(file_name)

# Vom extrage textul din fiecare pagină, păstrând informațiile despre font (pentru detectarea bold-ului)
lines_data = []

for page in doc:
    # Obținem reprezentarea paginii ca dicționar
    page_dict = page.get_text("dict")
    for block in page_dict["blocks"]:
        if "lines" in block:
            for line in block["lines"]:
                # Combinăm textul din toate fragmentele (spans) din linie
                line_text = "".join([span["text"] for span in line["spans"]]).strip()
                if not line_text:
                    continue
                # Păstrăm și lista de spanuri, care ne va oferi informații despre fonturi
                lines_data.append({"text": line_text, "spans": line["spans"]})

# Creăm o structură pentru întrebări cu opțiuni.
# Fiecare întrebare va fi un dicționar cu cheile:
#   'question_number', 'question_text' și 'options'
# 'options' va fi o listă de dicționare cu cheile:
#   'option_letter', 'option_text' și 'is_correct'

questions = []
current_question = None

# Modelele regex pentru detectarea unei întrebări și a unei opțiuni
question_pattern = re.compile(r"^\s*(\d+)\.\s*(.*)")
option_pattern = re.compile(r"^\s*([A-Z])\.\s*(.*)")

def is_line_bold(spans):
    """
    Funcția determină dacă o linie este afișată exclusiv în bold.
    Verifică fiecare span din linie: dacă fontul conține "Bold" (ex. "Times-Bold")
    atunci se consideră că fragmentul este bold.
    Returnează True dacă toate fragmentele non-goale sunt bold.
    """
    bold_flags = []
    for span in spans:
        if span["text"].strip() == "":
            continue
        if "Bold" in span["font"]:
            bold_flags.append(True)
        else:
            bold_flags.append(False)
    return all(bold_flags) if bold_flags else False

# Parcurgem liniile extrase pentru a construi baza de date
for line in lines_data:
    text = line["text"]
    spans = line["spans"]

    # Verificăm dacă linia începe cu modelul unei întrebări (ex. "1. Ce este ...")
    q_match = question_pattern.match(text)
    # Verificăm dacă linia începe cu modelul unei opțiuni (ex. "A. Răspuns ...")
    o_match = option_pattern.match(text)

    if q_match:
        # Am întâlnit o întrebare – dacă există o întrebare în curs, o salvăm anterior
        if current_question is not None:
            questions.append(current_question)
        q_number = q_match.group(1)
        q_text = q_match.group(2).strip()
        current_question = {
            "question_number": q_number,
            "question_text": q_text,
            "options": []
        }
    elif o_match and current_question is not None:
        # Am întâlnit o opțiune. Extragem litera, textul (fără prefixul de tip "A. ") și verificăm formatul bold
        letter = o_match.group(1)
        option_text = o_match.group(2).strip()
        is_correct = is_line_bold(spans)
        current_question["options"].append({
            "option_letter": letter,
            "option_text": option_text,
            "is_correct": is_correct
        })
    else:
        # Linia nu corespunde nici unui pattern de întrebare, nici unui pattern de opțiune.
        # Dacă încă nu s-a identificat nicio opțiune pentru întrebarea în curs, adăugăm linia la textul întrebării.
        # Altfel, considerăm că este o continuare a ultimei opțiuni.
        if current_question is not None:
            if len(current_question["options"]) == 0:
                current_question["question_text"] += " " + text
            else:
                last_option = current_question["options"][-1]
                last_option["option_text"] += " " + text

# După parcurgerea tuturor liniilor, adăugăm ultima întrebare (dacă există)
if current_question is not None:
    questions.append(current_question)

# Construim un DataFrame "flat" în care fiecare linie corespunde unei opțiuni
rows = []
for q in questions:
    q_no = q["question_number"]
    q_text = q["question_text"]
    for opt in q["options"]:
        rows.append({
            "question_number": q_no,
            "question_text": q_text,
            "option_letter": opt["option_letter"],
            "option_text": opt["option_text"],
            "is_correct": opt["is_correct"]
        })

df = pd.DataFrame(rows)
# print("Datele extrase:")
# print(df)
# df.to_csv("questions.csv", index=False)
# files.download("questions.csv")


Încarcă fișierul PDF cu grilele pentru examen:


Saving Grile Orto-Traumato 2022-2023.pdf to Grile Orto-Traumato 2022-2023 (1).pdf


In [48]:
#@title generator de grile
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import random

# Injectăm stilul CSS pentru a forța afișarea integrală a textului în etichetele widget
display(HTML('''
<style>
    .widget-label {
        white-space: pre-wrap !important;
        word-wrap: break-word !important;
        overflow: visible !important;
    }
</style>
'''))

# Filtrăm întrebările care au cel puțin 5 opțiuni și care conțin cel puțin 1 variantă corectă și 1 variantă greșită.
questions_valid = [
    q for q in questions
    if (len(q["options"]) >= 5) and (1 <= sum(1 for o in q["options"] if o["is_correct"]) < len(q["options"]))
]

if not questions_valid:
    print("Nu există grile care să îndeplinească condițiile (minim 5 opțiuni, cel puțin 1 corect și cel puțin 1 greșit).")
else:
    valid_question_numbers = sorted([int(q["question_number"]) for q in questions_valid if q["question_number"].isdigit()])
    min_q = valid_question_numbers[0] if valid_question_numbers else 1
    max_q = valid_question_numbers[-1] if valid_question_numbers else len(questions_valid)

# --- Interfața grafică ---

# Widget pentru afișarea întrebării, cu layout care permite text complet pe mai multe rânduri
question_text_widget = widgets.HTML(
    value="<div style='overflow: visible; white-space: pre-wrap; word-wrap: break-word;'><b>Întrebarea va apărea aici.</b></div>",
    layout=widgets.Layout(width='100%', height='auto')
)

# Cream 5 checkbox-uri pentru opțiuni și îi setăm layout-ul să se extindă pe tot spațiul orizontal
option_checkboxes = [widgets.Checkbox(value=False, description="") for _ in range(5)]
for cb in option_checkboxes:
    cb.layout = widgets.Layout(width='100%')

# Butoanele de acțiune și widget-ul pentru numărul grilei
verifica_button = widgets.Button(description="Verifică", button_style='info')
random_button = widgets.Button(description="Grilă Random", button_style='warning')
qnum_input = widgets.BoundedIntText(value=min_q, min=min_q, max=max_q, description="Nr grilă:")

output = widgets.Output()

# Variabile globale pentru întrebarea curentă și subsetul de opțiuni selectat
current_question = None
current_options = None

def choose_valid_subset(options, k=5, max_attempts=100):
    """
    Din lista totală de opțiuni, alege aleatoriu un subset de k opțiuni
    astfel încât numărul de variante corecte din acel subset să fie între 1 și 4.
    """
    for attempt in range(max_attempts):
        subset = random.sample(options, k)
        correct_count = sum(1 for o in subset if o["is_correct"])
        if 1 <= correct_count <= 4:
            return subset
    return None

def load_question(q):
    """
    Încarcă întrebarea q și alege aleatoriu 5 variante din toate opțiunile disponibile,
    astfel încât subsetul să respecte condiția (1-4 variante corecte).
    Se afișează doar textul întrebării și al opțiunilor, fără numărul grilei sau literele de prefix.
    """
    global current_question, current_options
    current_question = q
    subset = choose_valid_subset(q["options"], k=5)
    if subset is None:
        with output:
            clear_output()
            print("Nu s-a găsit un subset valid pentru această grilă. Încercați altă grilă.")
        return
    current_options = subset
    # Afișăm întrebarea complet, folosind un div cu white-space: pre-wrap
    question_text_widget.value = f"<div style='overflow: visible; white-space: pre-wrap; word-wrap: break-word;'><b>{q['question_text']}</b></div>"
    for i, opt in enumerate(current_options):
        option_checkboxes[i].description = f"{opt['option_text']}"
        option_checkboxes[i].value = False
        option_checkboxes[i].disabled = False
    output.clear_output()

def on_verifica_clicked(b):
    """
    La apăsarea butonului "Verifică", se generează un tabel HTML cu 5 coloane:
      1. "Grilă și răspuns": combinația număr grilă - litera variantei (a se prelua din baza de date)
      2. "Text": textul complet al opțiunii
      3. "Barem": afișează TRUE dacă opțiunea este corectă (din baza de date), FALSE dacă nu
      4. "Răspunsurile tale": afișează TRUE dacă utilizatorul a bifat, FALSE dacă nu
      5. "Rezultat": afișează ✔ verde dacă răspunsul utilizatorului corespunde cu baza de date; altfel ✖ roșu.
    Tabelul folosește stiluri CSS pentru a permite textului să se lipească pe mai multe rânduri.
    """
    with output:
        clear_output()
        result_html = f"<h3>Evaluare grilă #{current_question['question_number']}:</h3>"
        result_html += (
            "<table style='border-collapse: collapse; width: 100%; table-layout: fixed;'>"
            "<tr>"
            "<th style='border: 1px solid black; padding: 5px; word-wrap: break-word;'>Grilă și răspuns</th>"
            "<th style='border: 1px solid black; padding: 5px; word-wrap: break-word;'>Text</th>"
            "<th style='border: 1px solid black; padding: 5px; word-wrap: break-word;'>Barem</th>"
            "<th style='border: 1px solid black; padding: 5px; word-wrap: break-word;'>Răspunsurile tale</th>"
            "<th style='border: 1px solid black; padding: 5px; word-wrap: break-word;'>Rezultat</th>"
            "</tr>"
        )
        for i, opt in enumerate(current_options):
            letter = opt["option_letter"]
            text = opt["option_text"]
            user_selected = option_checkboxes[i].value
            correct = opt["is_correct"]

            grila_raspuns = f"Grilă #{current_question['question_number']} - {letter}"
            barem_value = "TRUE" if correct else "FALSE"
            user_value = "TRUE" if user_selected else "FALSE"
            rezultat_html = "<span style='color:green; font-weight:bold;'>✔</span>" if (correct == user_selected) else "<span style='color:red; font-weight:bold;'>✖</span>"

            result_html += (
                f"<tr>"
                f"<td style='border: 1px solid black; padding: 5px; word-wrap: break-word; white-space: normal;'>{grila_raspuns}</td>"
                f"<td style='border: 1px solid black; padding: 5px; word-wrap: break-word; white-space: normal;'>{text}</td>"
                f"<td style='border: 1px solid black; padding: 5px; text-align:center; word-wrap: break-word; white-space: normal;'>{barem_value}</td>"
                f"<td style='border: 1px solid black; padding: 5px; text-align:center; word-wrap: break-word; white-space: normal;'>{user_value}</td>"
                f"<td style='border: 1px solid black; padding: 5px; text-align:center; word-wrap: break-word; white-space: normal;'>{rezultat_html}</td>"
                f"</tr>"
            )
            option_checkboxes[i].disabled = True
        result_html += "</table>"
        display(HTML(result_html))

def on_random_clicked(b):
    """ Încarcă aleatoriu o grilă validă. """
    q = random.choice(questions_valid)
    load_question(q)

def on_qnum_submit(change):
    """ Când utilizatorul introduce manual un număr de grilă, se caută întrebarea respectivă. """
    if change["name"] == "value":
        num = change["new"]
        found = False
        for q in questions_valid:
            try:
                if int(q["question_number"]) == num:
                    load_question(q)
                    found = True
                    break
            except:
                continue
        if not found:
            with output:
                clear_output()
                print(f"Nu s-a găsit grila cu numărul: {num}")

random_button.on_click(on_random_clicked)
verifica_button.on_click(on_verifica_clicked)
qnum_input.observe(on_qnum_submit, names="value")

ui = widgets.VBox([
    widgets.HBox([random_button, qnum_input]),
    question_text_widget,
    widgets.VBox(option_checkboxes),
    verifica_button,
    output
])

display(ui)
